In [1]:
import sys
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse

In [2]:
class KalmanFilter:   ###kfmotionupdate
    def __init__(self, envmap, init_pose, motion_noise_stds):
        self.belief = multivariate_normal(mean=np.array([0.0, 0.0, 0.0]), 
                                        cov=np.diag([1e-10, 1e-10, 1e-10]))
        self.motion_noise_stds = motion_noise_stds #追加
        
    def motion_update(self, nu, omega, time): #追加
        if nu < 1e-10 and omega < 1e-10:
            return
        
        v = self.motion_noise_stds
        M = np.diag([v["nn"]**2*nu/time + v["no"]**2*omega/time, 
                     v["on"]**2*nu/time + v["oo"]**2*omega/time])
        
        A = time * np.array([[math.cos(self.belief.mean[2]), 0.0],
                             [math.sin(self.belief.mean[2]), 0.0],
                             [0.0, 1.0]])
        
        R = A.dot(M).dot(A.T)
        
        print(R)
        
    def draw(self, ax, elems):
        ###xy平面上での誤差楕円（3シグマ範囲）###
        eig_vals, eig_vec = np.linalg.eig(self.belief.cov[0:2, 0:2])
        ang = math.atan2(eig_vec[:,0][1], eig_vec[:,0][0])/math.pi*180
        e = Ellipse(self.belief.mean[0:2], width=3*eig_vals[0],height=3*eig_vals[1],
                       angle=ang, fill=False, color="blue", alpha=0.5)
        elems.append(ax.add_patch(e))

        ###θ方向の誤差の3シグマ範囲###
        x, y, c = self.belief.mean
        sigma3 = math.sqrt(self.belief.cov[2, 2])*3
        xs = [x + math.cos(c-sigma3), x, x + math.cos(c+sigma3)]
        ys = [y + math.sin(c-sigma3), y, y + math.sin(c+sigma3)]
        elems += ax.plot(xs, ys, color="blue", alpha=0.5)

In [3]:
class KfAgent(Agent):
    def __init__(self, time_interval, nu, omega, init_pose, envmap, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}):
        super().__init__(nu, omega)
        self.kf = KalmanFilter(envmap, init_pose, motion_noise_stds) #書き換え
        self.time_interval = time_interval
        
    def decision(self, observation=None): ###fig:kfdecision1
        self.kf.motion_update(self.nu, self.omega, self.time_interval)
        return self.nu, self.omega
        
    def draw(self, ax, elems): #追加
        self.kf.draw(ax, elems)

In [4]:
if __name__ == '__main__': 
    time_interval = 0.1
    world = World(30, time_interval) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    circling = KfAgent(time_interval, 0.2, 10.0/180*math.pi, np.array([0, 0, 0]).T, m)
    r = Robot(np.array([0, 0, 0]).T, sensor=Camera(m), agent=circling, color="red")
    world.append(r)

    #world.draw()                       # アニメーションさせるとき
    r.one_step(time_interval)  # アニメーションなしでデバッグするとき

[[ 0.00072202  0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.00103613]]
